<a href="https://colab.research.google.com/github/DaeSeokSong/image-processing/blob/feature%2FPreprocessing/Preprocessor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install**

# **Import**

In [ ]:
# Image processing
import cv2
import numpy as np

from google.colab.patches import cv2_imshow
from google.colab import output

# ETC
import os
import time
import math

from sklearn.model_selection import train_test_split

## Mount google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd /content/gdrive/MyDrive/Models/GAN_Scar
!ls -al

/content/gdrive/MyDrive/Models/GAN_Scar
total 10890
drwx------ 2 root root     4096 Sep 14 06:57  Dataset
-rw------- 1 root root    22891 Sep  1 12:04  Image_segmentation-Scar.ipynb
drwx------ 2 root root     4096 Aug 17 05:49  Log
-rw------- 1 root root    99753 Sep 14 08:48  Processor_PerformanceTester-Scar.ipynb
drwx------ 4 root root     4096 Sep 14 06:04 'Raw Dataset'
-rw------- 1 root root    16167 Sep  1 12:13  RawDataset_Processor-Scar.ipynb
-rw------- 1 root root    39995 Aug 15 11:40 'UNet architecture.PNG'
-rw------- 1 root root 10959279 Sep 14 07:49  Unet-Scar.ipynb


# **Grobal variable**

In [ ]:
# Path
MODEL_PATH = "/content/gdrive/MyDrive/Models/GAN_Scar"

RAW_DATA_PATH = "/Raw Dataset"

WOUND_TRAIN_PATH = "/Wound/train"
WOUND_TEST_PATH = "/Wound/test"
CVC_INPUT_PATH = "/CVC-clinicDB/Original"
CVC_GT_PATH = "/CVC-clinicDB/Ground Truth"

DATASET_PATH = "/Dataset"

IMAGES_PATH = '/images'
LABELS_PATH = '/labels'

# Image preprocess
NORM_INPUT_W_SIZE = 224
NORM_INPUT_H_SIZE = 224

# **Function**

In [ ]:
def imshow_waitkey_enter(image):
    cv2_imshow(image)

    time.sleep(0.5)
    
    input("Please press the Enter key to proceed\n")
    output.clear()

    pass

# **Make dir**

In [ ]:
# Create processed dataset dir
train_dir = os.path.join(MODEL_PATH + DATASET_PATH, 'train')
val_dir = os.path.join(MODEL_PATH + DATASET_PATH, 'val')
test_dir = os.path.join(MODEL_PATH + DATASET_PATH, 'test')

if not os.path.exists(train_dir):
    os.makedirs(train_dir)

if not os.path.exists(val_dir):
    os.makedirs(val_dir)

if not os.path.exists(test_dir):
    os.makedirs(test_dir)

<br>
<br>
<br>

# ***Preprocess Performace Compare Test***


---

<br>

***공통사항***
* 원본 이미지 크기 = 244 * 244
* epoch = 40
* batch = 8
> UNet batch size = 4 <br>
> Wound dataset batch size = 8
* lr = 1e-3
* Train:Val = 8:2
* Used model = Polar Res UNet++ (ISIC 2018 dice scroe rank no.1)

<br>

***주의사항***
* 이미지의 사이즈는 정규화 되어야 한다.
* Convolution에서 특징점을 잘 추출하기 때문에 색상영역이나, 스무딩/샤프닝 같은 필터 처리는 되려 성능저하 요소가 될 수 있다. 

<br>

***참고사항***
* CNN 대표 모델들의 네트워크 입력 사이즈는 224 * 224
* 이미지의 사이즈가 큰 경우에, Overlap-Tite(down sampling) 전략으로 한 이미지를 나눠서 학습시킨다.
* Polar Res UNet++ 의 input/output 사이즈는 [(384*512) == (y, x)](https://github.com/marinbenc/medical-polar-training/blob/main/datasets/lesion/make_dataset.py#L62)
> Dice score = 0.925 (Dice loss = 0.075)

<br>

***현재 결과***
* Resize 하면 loss가 높아져서 안 좋다.
> Resize가 아닌 Bordering을 통해 Size를 조정해야한다.
* Color나 Grayscale이나 성능차이가 안 난다.
> Grayscale이 학습이 빠르므로 Grayscale로 학습한다.

---




<br>
<br>
<br>

## [***Control Group***](https://www.nature.com/articles/s41598-020-78799-w)
> [Train code](https://github.com/uwm-bigdata/wound-segmentation/blob/de3b9c00974b065b2062fd5cd2efe670da1f8f51/train.py)
> <br>
> [Data generator](https://github.com/uwm-bigdata/wound-segmentation/blob/de3b9c00974b065b2062fd5cd2efe670da1f8f51/utils/io/data.py#L10)
> <br>
> <br>
> Raw dataset에 대한 전처리 없이, load 후 model에 들어가기 전에 Normalize
> <br>
> * 현재까지 대조군 성능을 넘은 실험군이 없다.

```
model = Deeplabv3(input_shape=(input_dim_x, input_dim_y, 3), classes=1)
```

### Performance

<br>

**Gray scale**
```
Input size =  torch.Size([8, 1, 224, 224])

TRAIN || EPOCH 0020 | BATCH  079 /  083 | DICE LOSS  0.1887
TRAIN || EPOCH 0020 | BATCH  080 /  083 | DICE LOSS  0.1890
TRAIN || EPOCH 0020 | BATCH  081 /  083 | DICE LOSS  0.1885
TRAIN || EPOCH 0020 | BATCH  082 /  083 | DICE LOSS  0.1872
TRAIN || EPOCH 0020 | BATCH  083 /  083 | DICE LOSS  0.1864

VALID || EPOCH 0020 | BATCH  017 /  021 | DICE_LOSS  0.1605
VALID || EPOCH 0020 | BATCH  018 /  021 | DICE_LOSS  0.1673
VALID || EPOCH 0020 | BATCH  019 /  021 | DICE_LOSS  0.1699
VALID || EPOCH 0020 | BATCH  020 /  021 | DICE_LOSS  0.1710
VALID || EPOCH 0020 | BATCH  021 /  021 | DICE_LOSS  0.1736
```

<br>

**Color**
```
Input size =  torch.Size([8, 3, 224, 224])

TRAIN || EPOCH 0020 | BATCH  079 /  083 | DICE LOSS  0.1855
TRAIN || EPOCH 0020 | BATCH  080 /  083 | DICE LOSS  0.1848
TRAIN || EPOCH 0020 | BATCH  081 /  083 | DICE LOSS  0.1838
TRAIN || EPOCH 0020 | BATCH  082 /  083 | DICE LOSS  0.1835
TRAIN || EPOCH 0020 | BATCH  083 /  083 | DICE LOSS  0.1824

VALID || EPOCH 0020 | BATCH  017 /  021 | DICE_LOSS  0.1781
VALID || EPOCH 0020 | BATCH  018 /  021 | DICE_LOSS  0.1831
VALID || EPOCH 0020 | BATCH  019 /  021 | DICE_LOSS  0.1798
VALID || EPOCH 0020 | BATCH  020 /  021 | DICE_LOSS  0.1789
VALID || EPOCH 0020 | BATCH  021 /  021 | DICE_LOSS  0.1865

Best loss in Train =  0.1824
Best loss in Validation =  0.1812
```

### Arrange dataset
> Not change anything about input image(scar)

In [ ]:
# Set load image dir path
train_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + IMAGES_PATH
train_label_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + LABELS_PATH
test_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + IMAGES_PATH
test_label_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + LABELS_PATH

# Load raw input images for Train
inputs_train = os.listdir(train_image_path)
inputs_train.sort()

# Load raw label images for Train
labels_train = os.listdir(train_label_path)
labels_train.sort()

# Load raw input images for Test
input_test = os.listdir(test_image_path)
input_test.sort()

# Load raw label images for Test
label_test = os.listdir(test_label_path)
label_test.sort()

# Split train:val:test = 8:2
input_train, input_val, label_train, label_val = train_test_split(inputs_train, labels_train, test_size=0.2, random_state=1)

# Merge train/val/test set
train_dataset = dict(zip(input_train, label_train))
val_dataset = dict(zip(input_val, label_val))
test_dataset = dict(zip(input_test, label_test))

# ================================ Train ================================
# Init image index
idx = 0

# Create preprocessed scar image
for input, label in train_dataset.items(): 
    # Access input image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access label image
    os.chdir(train_label_path)
    label_image = cv2.imread(label)

    # Save normalized image
    cv2.imwrite(os.path.join(train_dir, f'scar_{idx:03d}.png'), image)
    cv2.imwrite(os.path.join(train_dir, f'label_{idx:03d}.png'), label_image)
    idx += 1

# ============================== Validation ==============================
# Init image index
idx = 0

# Create preprocessed scar image
for input, label in val_dataset.items(): 
    # Access input image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access label image
    os.chdir(train_label_path)
    label_image = cv2.imread(label)

    # Save normalized image
    cv2.imwrite(os.path.join(val_dir, f'scar_{idx:03d}.png'), image)
    cv2.imwrite(os.path.join(val_dir, f'label_{idx:03d}.png'), label_image)
    idx += 1

# ================================= Test =================================
# Init image index
idx = 0

# Create preprocessed scar image
for input, label in test_dataset.items(): 
    # Access input image
    os.chdir(test_image_path)
    image = cv2.imread(input)

    # Access label image
    os.chdir(test_label_path)
    label_image = cv2.imread(label)

    # Save normalized image
    cv2.imwrite(os.path.join(test_dir, f'scar_{idx:03d}.png'), image)
    cv2.imwrite(os.path.join(test_dir, f'label_{idx:03d}.png'), label_image)
    idx += 1

## ***Experimental Group***

### Case 1

1. Delete Black Boundary(Padding)
2. Resize Scar Image to (224, 224)

### Case 2

1. Delete Black Boundary(Padding)
2. Pad the rest part to (224, 224) based on the middle of the image

### Case 3

1. Delete Black Boundary(Padding)
2. Resize Scar Image to (224, 224) in proportion to the maximum ratio
3. Pad the rest part to (224, 224) in the direction of right/down 

### Case 4

1. Delete Black Boundary(Padding)
2. Resize Scar Image to (224, 224) in proportion to the maximum ratio
3. Pad the rest part to (224, 224) based on the middle of the image

### Case 5

1. Delete Black Boundary(Padding)
2. Resize Scar Image to 224 based on the smaller size(row or column), keeping the proportion
3. Sampling by moving the kernel both on input and label, Then merge it to fit the label and input pair

<br>
<br>
<br>
<br>
<br>
<br>

# **Legacy System**

## 실험군 (Grayscale)

### Case 1

*적용사항*
1. Lab space의 l, a 채널 CLAHE Contrast 조정
2. BGR의 Green channel을 CLAHE Contrast 조정 후 input으로 사용

#### Performance

```
Input size =  torch.Size([8, 1, 224, 224])

TRAIN || EPOCH 0020 | BATCH  079 /  083 | DICE LOSS  0.2240
TRAIN || EPOCH 0020 | BATCH  080 /  083 | DICE LOSS  0.2236
TRAIN || EPOCH 0020 | BATCH  081 /  083 | DICE LOSS  0.2238
TRAIN || EPOCH 0020 | BATCH  082 /  083 | DICE LOSS  0.2234
TRAIN || EPOCH 0020 | BATCH  083 /  083 | DICE LOSS  0.2225

VALID || EPOCH 0020 | BATCH  017 /  021 | DICE_LOSS  0.2402
VALID || EPOCH 0020 | BATCH  018 /  021 | DICE_LOSS  0.2370
VALID || EPOCH 0020 | BATCH  019 /  021 | DICE_LOSS  0.2446
VALID || EPOCH 0020 | BATCH  020 /  021 | DICE_LOSS  0.2444
VALID || EPOCH 0020 | BATCH  021 /  021 | DICE_LOSS  0.2418
```

In [ ]:
# Set load image dir path
image_path = MODEL_PATH + RAW_TRAIN_SET_PATH + IMAGES_PATH
label_path = MODEL_PATH + RAW_TRAIN_SET_PATH + LABELS_PATH

# Load raw train images
train_files = os.listdir(image_path)
train_files.sort()

# Load raw label images
train_label_files = os.listdir(label_path)
train_label_files.sort()

# Divide train:val = 8:2
dataset_size = len(train_files)
train_size = int(dataset_size * 0.8)

# Create processed scar image for train
for idx in range(train_size): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    l_image = l_clahe.apply(l_image)
    a_image = a_clahe.apply(a_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)

    # Split each BGR
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

    # Save normalized image
    cv2.imwrite(os.path.join(train_dir, f'scar_{idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(train_dir, f'label_{idx:03d}.png'), label_image)

# Create processed scar image for validation
for val_idx, idx in enumerate(range(train_size, dataset_size)): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    l_image = l_clahe.apply(l_image)
    a_image = a_clahe.apply(a_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)

    # Split each BGR
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

    # Save normalized image
    cv2.imwrite(os.path.join(val_dir, f'scar_{val_idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(val_dir, f'label_{val_idx:03d}.png'), label_image)

#### Case 1-1

*변경된 적용사항*
1. Black boundary 제거
2. Lab space의 l, a 채널 CLAHE Contrast 조정
3. BGR의 Green channel을 CLAHE Contrast 조정
4. Right/Bottom으로 bordering to size (384, 512) == (y, x)

##### Performance

```
Input size =  torch.Size([8, 1, 384, 512])

TRAIN || EPOCH 0020 | BATCH  079 /  083 | DICE LOSS  0.2337
TRAIN || EPOCH 0020 | BATCH  080 /  083 | DICE LOSS  0.2344
TRAIN || EPOCH 0020 | BATCH  081 /  083 | DICE LOSS  0.2339
TRAIN || EPOCH 0020 | BATCH  082 /  083 | DICE LOSS  0.2340
TRAIN || EPOCH 0020 | BATCH  083 /  083 | DICE LOSS  0.2340

VALID || EPOCH 0020 | BATCH  017 /  021 | DICE_LOSS  0.2312
VALID || EPOCH 0020 | BATCH  018 /  021 | DICE_LOSS  0.2324
VALID || EPOCH 0020 | BATCH  019 /  021 | DICE_LOSS  0.2281
VALID || EPOCH 0020 | BATCH  020 /  021 | DICE_LOSS  0.2353
VALID || EPOCH 0020 | BATCH  021 /  021 | DICE_LOSS  0.2403
```

In [ ]:
# Set load image dir path
image_path = MODEL_PATH + RAW_TRAIN_SET_PATH + IMAGES_PATH
label_path = MODEL_PATH + RAW_TRAIN_SET_PATH + LABELS_PATH

# Load raw train images
train_files = os.listdir(image_path)
train_files.sort()

# Load raw label images
train_label_files = os.listdir(label_path)
train_label_files.sort()

# Divide train:val = 8:2
dataset_size = len(train_files)
train_size = int(dataset_size * 0.8)

# Create processed scar image for train
for idx in range(train_size): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

# ============================== Added code ==============================
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]
# =======================================================================

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    l_image = l_clahe.apply(l_image)
    a_image = a_clahe.apply(a_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)

    # Split each BGR
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

# ============================== Added code ==============================
    # Pad image
    pad_right = int((NORM_INPUT_W_SIZE - width))
    pad_bottom = int((NORM_INPUT_H_SIZE - height))

    green_image = cv2.copyMakeBorder(green_image,
                               0,
                               pad_bottom,
                               0,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    label_image = cv2.copyMakeBorder(label_image,
                               0,
                               pad_bottom,
                               0,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])
# =======================================================================

    # Save normalized image
    cv2.imwrite(os.path.join(train_dir, f'scar_{idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(train_dir, f'label_{idx:03d}.png'), label_image)

# Create processed scar image for validation
for val_idx, idx in enumerate(range(train_size, dataset_size)): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

# ============================== Added code ==============================
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]
# =======================================================================

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    l_image = l_clahe.apply(l_image)
    a_image = a_clahe.apply(a_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)

    # Split each BGR
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

# ============================== Added code ==============================
    # Pad image
    pad_right = int((NORM_INPUT_W_SIZE - width))
    pad_bottom = int((NORM_INPUT_H_SIZE - height))

    green_image = cv2.copyMakeBorder(green_image,
                               0,
                               pad_bottom,
                               0,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    label_image = cv2.copyMakeBorder(label_image,
                               0,
                               pad_bottom,
                               0,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])
# =======================================================================

    # Save normalized image
    cv2.imwrite(os.path.join(val_dir, f'scar_{val_idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(val_dir, f'label_{val_idx:03d}.png'), label_image)

### Case 2

*적용사항*
1. 불필요 Black boundary 제거
2. Lab space의 l, a 채널 CLAHE Contrast 조정
3. BGR의 Green channel을 CLAHE Contrast 조정
4. Resize input/output to 384*512 (y, x)
> UNet 아키텍쳐 입력 이미지 사이즈

#### Performance

```
Input size =  torch.Size([8, 1, 384, 512])

TRAIN || EPOCH 0020 | BATCH  079 /  083 | DICE LOSS  0.2960
TRAIN || EPOCH 0020 | BATCH  080 /  083 | DICE LOSS  0.2947
TRAIN || EPOCH 0020 | BATCH  081 /  083 | DICE LOSS  0.2943
TRAIN || EPOCH 0020 | BATCH  082 /  083 | DICE LOSS  0.2956
TRAIN || EPOCH 0020 | BATCH  083 /  083 | DICE LOSS  0.2950

VALID || EPOCH 0020 | BATCH  017 /  021 | DICE_LOSS  0.2981
VALID || EPOCH 0020 | BATCH  018 /  021 | DICE_LOSS  0.2936
VALID || EPOCH 0020 | BATCH  019 /  021 | DICE_LOSS  0.2851
VALID || EPOCH 0020 | BATCH  020 /  021 | DICE_LOSS  0.2860
VALID || EPOCH 0020 | BATCH  021 /  021 | DICE_LOSS  0.2872
```

In [ ]:
# Set load image dir path
image_path = MODEL_PATH + RAW_TRAIN_SET_PATH + IMAGES_PATH
label_path = MODEL_PATH + RAW_TRAIN_SET_PATH + LABELS_PATH

# Load raw train images
train_files = os.listdir(image_path)
train_files.sort()

# Load raw label images
train_label_files = os.listdir(label_path)
train_label_files.sort()

# Divide train:val = 8:2
dataset_size = len(train_files)
train_size = int(dataset_size * 0.8)

# Create processed scar image for train
for idx in range(train_size): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

# ============================== Added code ==============================
    if width > height: height = width
    else: width = height
# =======================================================================

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    l_image = l_clahe.apply(l_image)
    a_image = a_clahe.apply(a_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)

    # Split each BGR
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

# ============================== Added code ==============================
    # Resize image
    green_image = cv2.resize(green_image, 
                             (NORM_INPUT_W_SIZE, NORM_INPUT_H_SIZE),
                             interpolation=cv2.INTER_CUBIC)
    label_image = cv2.resize(label_image, 
                             (NORM_INPUT_W_SIZE, NORM_INPUT_H_SIZE),
                             interpolation=cv2.INTER_CUBIC)
# =======================================================================

    # Save normalized image
    cv2.imwrite(os.path.join(train_dir, f'scar_{idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(train_dir, f'label_{idx:03d}.png'), label_image)

# Create processed scar image for validation
for val_idx, idx in enumerate(range(train_size, dataset_size)): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

# ============================== Added code ==============================
    if width > height: height = width
    else: width = height
# =======================================================================

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    l_image = l_clahe.apply(l_image)
    a_image = a_clahe.apply(a_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)

    # Split each BGR
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

# ============================== Added code ==============================
    # Resize image
    green_image = cv2.resize(green_image, 
                             (NORM_INPUT_W_SIZE, NORM_INPUT_H_SIZE),
                             interpolation=cv2.INTER_CUBIC)
    label_image = cv2.resize(label_image, 
                             (NORM_INPUT_W_SIZE, NORM_INPUT_H_SIZE),
                             interpolation=cv2.INTER_CUBIC)
# =======================================================================

    # Save normalized image
    cv2.imwrite(os.path.join(val_dir, f'scar_{val_idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(val_dir, f'label_{val_idx:03d}.png'), label_image)

### Case 3 (Local SOTA)

*적용사항*
1. Black boundary 모두 제거
2. Lab space의 l, a 채널 CLAHE Contrast 조정
3. BGR의 Green channel을 CLAHE Contrast 조정
4. Bordering(cv2.copymakeborder CONSTANT) input/output to 384*512 (y, x)
> UNet 아키텍쳐 입력 이미지 사이즈

##### Performance

```
Input size =  torch.Size([8, 1, 384, 512])

TRAIN || EPOCH 0020 | BATCH  079 /  083 | DICE LOSS  0.2293
TRAIN || EPOCH 0020 | BATCH  080 /  083 | DICE LOSS  0.2289
TRAIN || EPOCH 0020 | BATCH  081 /  083 | DICE LOSS  0.2284
TRAIN || EPOCH 0020 | BATCH  082 /  083 | DICE LOSS  0.2279
TRAIN || EPOCH 0020 | BATCH  083 /  083 | DICE LOSS  0.2279

VALID || EPOCH 0020 | BATCH  017 /  021 | DICE_LOSS  0.2322
VALID || EPOCH 0020 | BATCH  018 /  021 | DICE_LOSS  0.2458
VALID || EPOCH 0020 | BATCH  019 /  021 | DICE_LOSS  0.2446
VALID || EPOCH 0020 | BATCH  020 /  021 | DICE_LOSS  0.2419
VALID || EPOCH 0020 | BATCH  021 /  021 | DICE_LOSS  0.2499

```

In [ ]:
# Set load image dir path
image_path = RAW_DATA_PATH + WOUND_TRAIN_PATH
label_path = RAW_DATA_PATH + WOUND_TEST_PATH

# Load raw train images
train_files = os.listdir(image_path)
train_files.sort()

# Load raw label images
train_label_files = os.listdir(label_path)
train_label_files.sort()

# Divide train:val:test = 6:2:2
dataset_size = len(train_files)
train_size = int(dataset_size * 0.8)

# Create processed scar image for train
for idx in range(train_size):
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    l_image = l_clahe.apply(l_image)
    a_image = a_clahe.apply(a_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)

    # Split each BGR
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

# ============================== Added code ==============================
    # Pad image
    pad_width = int((NORM_INPUT_W_SIZE - width) / 2)
    pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width

    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)
        
    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_H_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_H_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_W_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_W_SIZE - total_width)

    green_image = cv2.copyMakeBorder(green_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    label_image = cv2.copyMakeBorder(label_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])
# =======================================================================

    # Save normalized image
    cv2.imwrite(os.path.join(train_dir, f'scar_{idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(train_dir, f'label_{idx:03d}.png'), label_image)

# Create processed scar image for validation
for val_idx, idx in enumerate(range(train_size, dataset_size)): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    l_image = l_clahe.apply(l_image)
    a_image = a_clahe.apply(a_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)

    # Split each BGR
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

# ============================== Added code ==============================
    # Pad image
    pad_width = int((NORM_INPUT_W_SIZE - width) / 2)
    pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width

    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)
        
    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_H_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_H_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_W_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_W_SIZE - total_width)

    green_image = cv2.copyMakeBorder(green_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    label_image = cv2.copyMakeBorder(label_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])
# =======================================================================

    # Save normalized image
    cv2.imwrite(os.path.join(val_dir, f'scar_{val_idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(val_dir, f'label_{val_idx:03d}.png'), label_image)

#### Case 3-1

3. BGR의 Green channel을 CLAHE Contrast 조정
> 변경) Blue constrast down, Red/Green contrast up

##### Performance



```
Input size =  torch.Size([8, 1, 384, 512])

TRAIN || EPOCH 0020 | BATCH  079 /  083 | DICE LOSS  0.2519
TRAIN || EPOCH 0020 | BATCH  080 /  083 | DICE LOSS  0.2507
TRAIN || EPOCH 0020 | BATCH  081 /  083 | DICE LOSS  0.2511
TRAIN || EPOCH 0020 | BATCH  082 /  083 | DICE LOSS  0.2521
TRAIN || EPOCH 0020 | BATCH  083 /  083 | DICE LOSS  0.2513

VALID || EPOCH 0020 | BATCH  017 /  021 | DICE_LOSS  0.3144
VALID || EPOCH 0020 | BATCH  018 /  021 | DICE_LOSS  0.3115
VALID || EPOCH 0020 | BATCH  019 /  021 | DICE_LOSS  0.3074
VALID || EPOCH 0020 | BATCH  020 /  021 | DICE_LOSS  0.3134
VALID || EPOCH 0020 | BATCH  021 /  021 | DICE_LOSS  0.3137

Best loss in Train =  0.2499
Best loss in Validation =  0.2536
```



In [ ]:
# Set load image dir path
image_path = MODEL_PATH + RAW_TRAIN_SET_PATH + IMAGES_PATH
label_path = MODEL_PATH + RAW_TRAIN_SET_PATH + LABELS_PATH

# Load raw train images
train_files = os.listdir(image_path)
train_files.sort()

# Load raw label images
train_label_files = os.listdir(label_path)
train_label_files.sort()

# Divide train:val = 8:2
dataset_size = len(train_files)
train_size = int(dataset_size * 0.8)

# Create processed scar image for train
for idx in range(train_size): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    l_image = l_clahe.apply(l_image)
    a_image = a_clahe.apply(a_image)

# ============================ Modified code ============================
    enhanced_lab = cv2.merge((l_image, a_image, b_image))
    enhanced_bgr = cv2.cvtColor(enhanced_lab, cv2.COLOR_LAB2BGR)

    # Split each BGR
    blue_image, green_image, red_image = cv2.split(enhanced_bgr)

    b_alpha = -1.5
    g_alpha = 0.5
    r_alpha = 1.5

    blue_image = np.clip((1 + b_alpha)*blue_image - (128 * b_alpha), 0, 255).astype(np.uint8)
    green_image = np.clip((1 + g_alpha)*green_image - (128 * g_alpha), 0, 255).astype(np.uint8)
    red_image = np.clip((1 + r_alpha)*red_image - (128 * r_alpha), 0, 255).astype(np.uint8)

    enhanced_bgr = cv2.merge((blue_image, green_image, red_image))
    input_image = cv2.cvtColor(enhanced_bgr, cv2.COLOR_BGR2GRAY)
# =======================================================================

    # Pad image
    pad_width = int((NORM_INPUT_W_SIZE - width) / 2)
    pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width

    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)
        
    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_H_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_H_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_W_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_W_SIZE - total_width)

    input_image = cv2.copyMakeBorder(input_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    label_image = cv2.copyMakeBorder(label_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    cv2_imshow(input_image)
    imshow_waitkey_enter(label_image)
    
    # Save normalized image
    #cv2.imwrite(os.path.join(train_dir, f'scar_{idx:03d}.png'), input_image)
    #cv2.imwrite(os.path.join(train_dir, f'label_{idx:03d}.png'), label_image)

# Create processed scar image for validation
for val_idx, idx in enumerate(range(train_size, dataset_size)): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    l_image = l_clahe.apply(l_image)
    a_image = a_clahe.apply(a_image)

# ============================ Modified code ============================
    enhanced_lab = cv2.merge((l_image, a_image, b_image))
    enhanced_bgr = cv2.cvtColor(enhanced_lab, cv2.COLOR_LAB2BGR)

    # Split each BGR
    blue_image, green_image, red_image = cv2.split(enhanced_bgr)

    b_alpha = -0.5
    g_alpha = 2.5
    r_alpha = 1.5

    blue_image = np.clip((1 + b_alpha)*blue_image - (128 * b_alpha), 0, 255).astype(np.uint8)
    green_image = np.clip((1 + g_alpha)*green_image - (128 * g_alpha), 0, 255).astype(np.uint8)
    red_image = np.clip((1 + r_alpha)*red_image - (128 * r_alpha), 0, 255).astype(np.uint8)

    enhanced_bgr = cv2.merge((blue_image, green_image, red_image))
    gray_input = cv2.cvtColor(enhanced_bgr, cv2.COLOR_BGR2GRAY)
# =======================================================================

    # Pad image
    pad_width = int((NORM_INPUT_W_SIZE - width) / 2)
    pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width

    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)
        
    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_H_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_H_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_W_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_W_SIZE - total_width)

    gray_input = cv2.copyMakeBorder(gray_input,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    label_image = cv2.copyMakeBorder(label_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Save normalized image
    cv2.imwrite(os.path.join(val_dir, f'scar_{val_idx:03d}.png'), gray_input)
    cv2.imwrite(os.path.join(val_dir, f'label_{val_idx:03d}.png'), label_image)

#### Case 3-2

##### Blurring (Smoothing)
> Border 전에 양방향 필터 블러링(스무딩) 단계 추가

```
Input size =  torch.Size([8, 1, 384, 512])

TRAIN || EPOCH 0020 | BATCH  079 /  083 | DICE LOSS  0.2479
TRAIN || EPOCH 0020 | BATCH  080 /  083 | DICE LOSS  0.2488
TRAIN || EPOCH 0020 | BATCH  081 /  083 | DICE LOSS  0.2475
TRAIN || EPOCH 0020 | BATCH  082 /  083 | DICE LOSS  0.2472
TRAIN || EPOCH 0020 | BATCH  083 /  083 | DICE LOSS  0.2485

VALID || EPOCH 0020 | BATCH  017 /  021 | DICE_LOSS  0.3442
VALID || EPOCH 0020 | BATCH  018 /  021 | DICE_LOSS  0.3530
VALID || EPOCH 0020 | BATCH  019 /  021 | DICE_LOSS  0.3493
VALID || EPOCH 0020 | BATCH  020 /  021 | DICE_LOSS  0.3476
VALID || EPOCH 0020 | BATCH  021 /  021 | DICE_LOSS  0.3434

Best loss in Train =  0.2429
Best loss in Validation =  0.2410
```

In [ ]:
# Set load image dir path
image_path = MODEL_PATH + RAW_TRAIN_SET_PATH + IMAGES_PATH
label_path = MODEL_PATH + RAW_TRAIN_SET_PATH + LABELS_PATH

# Load raw train images
train_files = os.listdir(image_path)
train_files.sort()

# Load raw label images
train_label_files = os.listdir(label_path)
train_label_files.sort()

# Divide train:val = 8:2
dataset_size = len(train_files)
train_size = int(dataset_size * 0.8)

# Create processed scar image for train
for idx in range(train_size): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    l_image = l_clahe.apply(l_image)
    a_image = a_clahe.apply(a_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)

    # Split each BGR
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

# ============================== Added code ==============================
    # Bluring
    green_image = cv2.bilateralFilter(green_image, -1, 75, 75)
# =======================================================================

    # Pad image
    pad_width = int((NORM_INPUT_W_SIZE - width) / 2)
    pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width

    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)
        
    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_H_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_H_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_W_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_W_SIZE - total_width)

    green_image = cv2.copyMakeBorder(green_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    label_image = cv2.copyMakeBorder(label_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Save normalized image
    cv2.imwrite(os.path.join(train_dir, f'scar_{idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(train_dir, f'label_{idx:03d}.png'), label_image)

# Create processed scar image for validation
for val_idx, idx in enumerate(range(train_size, dataset_size)): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    l_image = l_clahe.apply(l_image)
    a_image = a_clahe.apply(a_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)

    # Split each BGR
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

# ============================== Added code ==============================
    # Bluring
    green_image = cv2.bilateralFilter(green_image, -1, 75, 75)
# =======================================================================

    # Pad image
    pad_width = int((NORM_INPUT_W_SIZE - width) / 2)
    pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width

    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)
        
    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_H_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_H_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_W_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_W_SIZE - total_width)

    green_image = cv2.copyMakeBorder(green_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    label_image = cv2.copyMakeBorder(label_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Save normalized image
    cv2.imwrite(os.path.join(val_dir, f'scar_{val_idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(val_dir, f'label_{val_idx:03d}.png'), label_image)

##### Sharpening
> Border 전에 샤프닝 단계 추가

```
Input size =  torch.Size([8, 1, 384, 512])

TRAIN || EPOCH 0020 | BATCH  079 /  083 | DICE LOSS  0.2186
TRAIN || EPOCH 0020 | BATCH  080 /  083 | DICE LOSS  0.2180
TRAIN || EPOCH 0020 | BATCH  081 /  083 | DICE LOSS  0.2177
TRAIN || EPOCH 0020 | BATCH  082 /  083 | DICE LOSS  0.2180
TRAIN || EPOCH 0020 | BATCH  083 /  083 | DICE LOSS  0.2181

VALID || EPOCH 0020 | BATCH  017 /  021 | DICE_LOSS  0.3035
VALID || EPOCH 0020 | BATCH  018 /  021 | DICE_LOSS  0.3050
VALID || EPOCH 0020 | BATCH  019 /  021 | DICE_LOSS  0.3037
VALID || EPOCH 0020 | BATCH  020 /  021 | DICE_LOSS  0.2996
VALID || EPOCH 0020 | BATCH  021 /  021 | DICE_LOSS  0.3051

Best loss in Train =  0.2181
Best loss in Validation =  0.2431
```

In [ ]:
# Set load image dir path
image_path = MODEL_PATH + RAW_TRAIN_SET_PATH + IMAGES_PATH
label_path = MODEL_PATH + RAW_TRAIN_SET_PATH + LABELS_PATH

# Load raw train images
train_files = os.listdir(image_path)
train_files.sort()

# Load raw label images
train_label_files = os.listdir(label_path)
train_label_files.sort()

# Divide train:val = 8:2
dataset_size = len(train_files)
train_size = int(dataset_size * 0.8)

# Create processed scar image for train
for idx in range(train_size): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    l_image = l_clahe.apply(l_image)
    a_image = a_clahe.apply(a_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)

    # Split each BGR
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

# ============================== Added code ==============================
    # Sharpening
    sharp_kernel = np.array([[0, -1, 0],
                             [-1, 9, -1],
                             [0, -1, 0]])
    sharp_image = cv2.filter2D(green_image, -1, sharp_kernel)

    green_image = cv2.addWeighted(green_image, 0.75, sharp_image, 0.25, 1)
# =======================================================================

    # Pad image
    pad_width = int((NORM_INPUT_W_SIZE - width) / 2)
    pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width

    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)
        
    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_H_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_H_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_W_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_W_SIZE - total_width)

    green_image = cv2.copyMakeBorder(green_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    label_image = cv2.copyMakeBorder(label_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])
    
    # Save normalized image
    cv2.imwrite(os.path.join(train_dir, f'scar_{idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(train_dir, f'label_{idx:03d}.png'), label_image)

# Create processed scar image for validation
for val_idx, idx in enumerate(range(train_size, dataset_size)): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    l_image = l_clahe.apply(l_image)
    a_image = a_clahe.apply(a_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)

    # Split each BGR
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

# ============================== Added code ==============================
    # Sharpening
    sharp_kernel = np.array([[0, -1, 0],
                             [-1, 9, -1],
                             [0, -1, 0]])
    sharp_image = cv2.filter2D(green_image, -1, sharp_kernel)

    green_image = cv2.addWeighted(green_image, 0.75, sharp_image, 0.25, 1)
# =======================================================================

    # Pad image
    pad_width = int((NORM_INPUT_W_SIZE - width) / 2)
    pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width

    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)
        
    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_H_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_H_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_W_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_W_SIZE - total_width)

    green_image = cv2.copyMakeBorder(green_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    label_image = cv2.copyMakeBorder(label_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Save normalized image
    cv2.imwrite(os.path.join(val_dir, f'scar_{val_idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(val_dir, f'label_{val_idx:03d}.png'), label_image)

##### Blurring (Smoothing) + Sharpening
> Bordering 전에 스무딩 후 샤프닝 단계 추가



```
Input size =  torch.Size([8, 1, 384, 512])

TRAIN || EPOCH 0020 | BATCH  078 /  083 | DICE LOSS  0.2372
TRAIN || EPOCH 0020 | BATCH  079 /  083 | DICE LOSS  0.2362
TRAIN || EPOCH 0020 | BATCH  080 /  083 | DICE LOSS  0.2357
TRAIN || EPOCH 0020 | BATCH  081 /  083 | DICE LOSS  0.2344
TRAIN || EPOCH 0020 | BATCH  082 /  083 | DICE LOSS  0.2346

VALID || EPOCH 0020 | BATCH  017 /  021 | DICE_LOSS  0.2690
VALID || EPOCH 0020 | BATCH  018 /  021 | DICE_LOSS  0.2643
VALID || EPOCH 0020 | BATCH  019 /  021 | DICE_LOSS  0.2591
VALID || EPOCH 0020 | BATCH  020 /  021 | DICE_LOSS  0.2603
VALID || EPOCH 0020 | BATCH  021 /  021 | DICE_LOSS  0.2716

Best loss in Train =  0.2346
Best loss in Validation =  0.2500
```



In [ ]:
# Set load image dir path
image_path = MODEL_PATH + RAW_TRAIN_SET_PATH + IMAGES_PATH
label_path = MODEL_PATH + RAW_TRAIN_SET_PATH + LABELS_PATH

# Load raw train images
train_files = os.listdir(image_path)
train_files.sort()

# Load raw label images
train_label_files = os.listdir(label_path)
train_label_files.sort()

# Divide train:val = 8:2
dataset_size = len(train_files)
train_size = int(dataset_size * 0.8)

# Create processed scar image for train
for idx in range(train_size): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    l_image = l_clahe.apply(l_image)
    a_image = a_clahe.apply(a_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)

    # Split each BGR
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

# ============================== Added code ==============================
    # Bluring
    green_image = cv2.bilateralFilter(green_image, -1, 75, 75)

    # Sharpening
    sharp_kernel = np.array([[0, -1, 0],
                             [-1, 9, -1],
                             [0, -1, 0]])
    sharp_image = cv2.filter2D(green_image, -1, sharp_kernel)

    green_image = cv2.addWeighted(green_image, 0.75, sharp_image, 0.25, 1)
# =======================================================================

    # Pad image
    pad_width = int((NORM_INPUT_W_SIZE - width) / 2)
    pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width

    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)
        
    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_H_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_H_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_W_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_W_SIZE - total_width)

    green_image = cv2.copyMakeBorder(green_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    label_image = cv2.copyMakeBorder(label_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Save normalized image
    cv2.imwrite(os.path.join(train_dir, f'scar_{idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(train_dir, f'label_{idx:03d}.png'), label_image)

# Create processed scar image for validation
for val_idx, idx in enumerate(range(train_size, dataset_size)): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    l_image = l_clahe.apply(l_image)
    a_image = a_clahe.apply(a_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)

    # Split each BGR
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

# ============================== Added code ==============================
    # Bluring
    green_image = cv2.bilateralFilter(green_image, -1, 75, 75)

    # Sharpening
    sharp_kernel = np.array([[0, -1, 0],
                             [-1, 9, -1],
                             [0, -1, 0]])
    sharp_image = cv2.filter2D(green_image, -1, sharp_kernel)

    green_image = cv2.addWeighted(green_image, 0.75, sharp_image, 0.25, 1)
# =======================================================================

    # Pad image
    pad_width = int((NORM_INPUT_W_SIZE - width) / 2)
    pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width

    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)
        
    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_H_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_H_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_W_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_W_SIZE - total_width)

    green_image = cv2.copyMakeBorder(green_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    label_image = cv2.copyMakeBorder(label_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Save normalized image
    cv2.imwrite(os.path.join(val_dir, f'scar_{val_idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(val_dir, f'label_{val_idx:03d}.png'), label_image)

### Case 4

*적용사항*
1. Black boundary 모두 제거
2. Lab space의 l, a 채널 CLAHE Contrast 조정
3. BGR의 Green channel을 CLAHE Contrast 조정
4. Resize input/label width to 384 or height to 512 (비율 유지)
5. Bordering(cv2.copymakeborder CONSTANT) input/output to 384*512 (y, x)
> UNet 아키텍쳐 입력 이미지 사이즈

#### Performance

```
Input size =  torch.Size([8, 1, 384, 512])

TRAIN || EPOCH 0020 | BATCH  079 /  083 | DICE LOSS  0.2895
TRAIN || EPOCH 0020 | BATCH  080 /  083 | DICE LOSS  0.2880
TRAIN || EPOCH 0020 | BATCH  081 /  083 | DICE LOSS  0.2888
TRAIN || EPOCH 0020 | BATCH  082 /  083 | DICE LOSS  0.2885
TRAIN || EPOCH 0020 | BATCH  083 /  083 | DICE LOSS  0.2902

VALID || EPOCH 0020 | BATCH  017 /  021 | DICE_LOSS  0.2928
VALID || EPOCH 0020 | BATCH  018 /  021 | DICE_LOSS  0.2864
VALID || EPOCH 0020 | BATCH  019 /  021 | DICE_LOSS  0.2852
VALID || EPOCH 0020 | BATCH  020 /  021 | DICE_LOSS  0.2823
VALID || EPOCH 0020 | BATCH  021 /  021 | DICE_LOSS  0.2844
```

In [ ]:
# Set load image dir path
image_path = MODEL_PATH + RAW_TRAIN_SET_PATH + IMAGES_PATH
label_path = MODEL_PATH + RAW_TRAIN_SET_PATH + LABELS_PATH

# Load raw train images
train_files = os.listdir(image_path)
train_files.sort()

# Load raw label images
train_label_files = os.listdir(label_path)
train_label_files.sort()

# Divide train:val = 8:2
dataset_size = len(train_files)
train_size = int(dataset_size * 0.8)

# Create processed scar image for train
for idx in range(train_size): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    l_image = l_clahe.apply(l_image)
    a_image = a_clahe.apply(a_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)

    # Split each BGR
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

# ============================== Added code ==============================
    # Resize images
    resize_scale_w = NORM_INPUT_W_SIZE / width
    resize_scale_h = NORM_INPUT_H_SIZE / height

    resize_criteria = -1
    if resize_scale_w < resize_scale_h:
        resize_scale = resize_scale_w
        resize_criteria = 0
    else:
        resize_scale = resize_scale_h
        resize_criteria = 1

    green_image = cv2.resize(green_image,
                             (0, 0),
                             fx=resize_scale,
                             fy=resize_scale,
                             interpolation=cv2.INTER_CUBIC)
    label_image = cv2.resize(label_image, 
                             (0, 0),
                             fx=resize_scale,
                             fy=resize_scale,
                             interpolation=cv2.INTER_CUBIC)
 
    width = round(width * resize_scale)
    height = round(height * resize_scale)
# =======================================================================

# ============================ Modified code ============================
    # Pad image
    pad_top = 0
    pad_bottom = 0
    pad_left = 0
    pad_right = 0

    if resize_criteria == 0:
        pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

        pad_top = pad_height
        pad_bottom = pad_height
        
        total_height = height + (pad_height * 2)

        if total_height > NORM_INPUT_H_SIZE:
            pad_top = pad_top - (total_height - NORM_INPUT_H_SIZE)
        elif total_height < NORM_INPUT_H_SIZE:
            pad_bottom = pad_bottom + (NORM_INPUT_H_SIZE - total_height)

    elif resize_criteria == 1:
        pad_width = int((NORM_INPUT_W_SIZE - width) / 2)

        pad_left = pad_width
        pad_right = pad_width

        total_width = width + (pad_width * 2)

        if total_width > NORM_INPUT_W_SIZE:
            pad_left = pad_left - (total_width - NORM_INPUT_W_SIZE)
        elif total_width < NORM_INPUT_W_SIZE:
            pad_right = pad_right + (NORM_INPUT_W_SIZE - total_width)

    else:
        print("Resize Error, [ocurred index] = ", idx)
        break
    
    green_image = cv2.copyMakeBorder(green_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    label_image = cv2.copyMakeBorder(label_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])
# =======================================================================
    
    # Save normalized image
    cv2.imwrite(os.path.join(train_dir, f'scar_{idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(train_dir, f'label_{idx:03d}.png'), label_image)

# Create processed scar image for validation
for val_idx, idx in enumerate(range(train_size, dataset_size)): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    l_image = l_clahe.apply(l_image)
    a_image = a_clahe.apply(a_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)
    
    # Split each BGR
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

# ============================== Added code ==============================
    # Resize images
    resize_scale_w = NORM_INPUT_W_SIZE / width
    resize_scale_h = NORM_INPUT_H_SIZE / height

    resize_criteria = -1
    if resize_scale_w < resize_scale_h:
        resize_scale = resize_scale_w
        resize_criteria = 0
    else:
        resize_scale = resize_scale_h
        resize_criteria = 1

    green_image = cv2.resize(green_image,
                             (0, 0),
                             fx=resize_scale,
                             fy=resize_scale,
                             interpolation=cv2.INTER_CUBIC)
    label_image = cv2.resize(label_image, 
                             (0, 0),
                             fx=resize_scale,
                             fy=resize_scale,
                             interpolation=cv2.INTER_CUBIC)
 
    width = round(width * resize_scale)
    height = round(height * resize_scale)
# =======================================================================

# ============================ Modified code ============================
    # Pad image
    pad_top = 0
    pad_bottom = 0
    pad_left = 0
    pad_right = 0

    if resize_criteria == 0:
        pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

        pad_top = pad_height
        pad_bottom = pad_height
        
        total_height = height + (pad_height * 2)

        if total_height > NORM_INPUT_H_SIZE:
            pad_top = pad_top - (total_height - NORM_INPUT_H_SIZE)
        elif total_height < NORM_INPUT_H_SIZE:
            pad_bottom = pad_bottom + (NORM_INPUT_H_SIZE - total_height)

    elif resize_criteria == 1:
        pad_width = int((NORM_INPUT_W_SIZE - width) / 2)

        pad_left = pad_width
        pad_right = pad_width

        total_width = width + (pad_width * 2)

        if total_width > NORM_INPUT_W_SIZE:
            pad_left = pad_left - (total_width - NORM_INPUT_W_SIZE)
        elif total_width < NORM_INPUT_W_SIZE:
            pad_right = pad_right + (NORM_INPUT_W_SIZE - total_width)

    else:
        print("Resize Error, [ocurred index] = ", idx)
        break
    
    green_image = cv2.copyMakeBorder(green_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    label_image = cv2.copyMakeBorder(label_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])
# =======================================================================
    
    # Save normalized image
    cv2.imwrite(os.path.join(val_dir, f'scar_{val_idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(val_dir, f'label_{val_idx:03d}.png'), label_image)

### Case 5

*적용사항*
1. Black boundary 모두 제거
2. Lab space의 l, a 채널 CLAHE Contrast 조정
3. K-Means(k=3)를 이용한 이미지 분류
4. Resize input/label width to 384 or height to 512 (비율 유지)
5. Bordering(cv2.copymakeborder CONSTANT) input/output to 384*512 (y, x)
> UNet 아키텍쳐 입력 이미지 사이즈

#### Performance

```
Input size =  torch.Size([8, 1, 384, 512])

TRAIN || EPOCH 0020 | BATCH  079 /  083 | DICE LOSS  0.3189
TRAIN || EPOCH 0020 | BATCH  080 /  083 | DICE LOSS  0.3178
TRAIN || EPOCH 0020 | BATCH  081 /  083 | DICE LOSS  0.3177
TRAIN || EPOCH 0020 | BATCH  082 /  083 | DICE LOSS  0.3207
TRAIN || EPOCH 0020 | BATCH  083 /  083 | DICE LOSS  0.3198

VALID || EPOCH 0020 | BATCH  017 /  021 | DICE_LOSS  0.3028
VALID || EPOCH 0020 | BATCH  018 /  021 | DICE_LOSS  0.2980
VALID || EPOCH 0020 | BATCH  019 /  021 | DICE_LOSS  0.3053
VALID || EPOCH 0020 | BATCH  020 /  021 | DICE_LOSS  0.3081
VALID || EPOCH 0020 | BATCH  021 /  021 | DICE_LOSS  0.3027
```

In [ ]:
# Set load image dir path
image_path = MODEL_PATH + RAW_TRAIN_SET_PATH + IMAGES_PATH
label_path = MODEL_PATH + RAW_TRAIN_SET_PATH + LABELS_PATH

# Load raw train images
train_files = os.listdir(image_path)
train_files.sort()

# Load raw label images
train_label_files = os.listdir(label_path)
train_label_files.sort()

# Divide train:val = 8:2
dataset_size = len(train_files)
train_size = int(dataset_size * 0.8)

# Create processed scar image for train
for idx in range(train_size): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    l_image = l_clahe.apply(l_image)
    a_image = a_clahe.apply(a_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)
    
# ============================== Added code ==============================
    # Clustering use K-means
    cluster_data = enhanced_image.reshape((-1, 3)).astype(np.float32)
    k = 3
    criteria = (cv2.TERM_CRITERIA_EPS|cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0) # 최대 10번 반복하고 1픽셀 이하로 움직이면 종료
    ret, c_label, center = cv2.kmeans(cluster_data, 
                                    k, 
                                    None, 
                                    criteria,
                                    10,
                                    cv2.KMEANS_RANDOM_CENTERS)
    
    center = np.uint8(center)
    kmeans_image = center[c_label.flatten()]
    kmeans_image = kmeans_image.reshape((enhanced_image.shape))
    kmeans_image = cv2.cvtColor(kmeans_image, cv2.COLOR_BGR2GRAY)
# =======================================================================

# ============================ Modified code ============================
    # Resize images
    resize_scale_w = NORM_INPUT_W_SIZE / width
    resize_scale_h = NORM_INPUT_H_SIZE / height

    resize_criteria = -1
    if resize_scale_w < resize_scale_h:
        resize_scale = resize_scale_w
        resize_criteria = 0
    else:
        resize_scale = resize_scale_h
        resize_criteria = 1

    kmeans_image = cv2.resize(kmeans_image,
                             (0, 0),
                             fx=resize_scale,
                             fy=resize_scale,
                             interpolation=cv2.INTER_CUBIC)
    label_image = cv2.resize(label_image, 
                             (0, 0),
                             fx=resize_scale,
                             fy=resize_scale,
                             interpolation=cv2.INTER_CUBIC)
 
    width = round(width * resize_scale)
    height = round(height * resize_scale)
# =======================================================================

    # Pad image
    pad_top = 0
    pad_bottom = 0
    pad_left = 0
    pad_right = 0

    if resize_criteria == 0:
        pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

        pad_top = pad_height
        pad_bottom = pad_height
        
        total_height = height + (pad_height * 2)

        if total_height > NORM_INPUT_H_SIZE:
            pad_top = pad_top - (total_height - NORM_INPUT_H_SIZE)
        elif total_height < NORM_INPUT_H_SIZE:
            pad_bottom = pad_bottom + (NORM_INPUT_H_SIZE - total_height)

    elif resize_criteria == 1:
        pad_width = int((NORM_INPUT_W_SIZE - width) / 2)

        pad_left = pad_width
        pad_right = pad_width

        total_width = width + (pad_width * 2)

        if total_width > NORM_INPUT_W_SIZE:
            pad_left = pad_left - (total_width - NORM_INPUT_W_SIZE)
        elif total_width < NORM_INPUT_W_SIZE:
            pad_right = pad_right + (NORM_INPUT_W_SIZE - total_width)

    else:
        print("Resize Error, [ocurred index] = ", idx)
        break
    
    kmeans_image = cv2.copyMakeBorder(kmeans_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    label_image = cv2.copyMakeBorder(label_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])
    
    # Save normalized image
    cv2.imwrite(os.path.join(train_dir, f'scar_{idx:03d}.png'), kmeans_image)
    cv2.imwrite(os.path.join(train_dir, f'label_{idx:03d}.png'), label_image)

# Create processed scar image for validation
for val_idx, idx in enumerate(range(train_size, dataset_size)): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    l_image = l_clahe.apply(l_image)
    a_image = a_clahe.apply(a_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)
    
# ============================== Added code ==============================
    # Clustering use K-means
    cluster_data = enhanced_image.reshape((-1, 3)).astype(np.float32)
    k = 3
    criteria = (cv2.TERM_CRITERIA_EPS|cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0) # 최대 10번 반복하고 1픽셀 이하로 움직이면 종료
    ret, c_label, center = cv2.kmeans(cluster_data, 
                                    k, 
                                    None, 
                                    criteria,
                                    10,
                                    cv2.KMEANS_RANDOM_CENTERS)
    
    center = np.uint8(center)
    kmeans_image = center[c_label.flatten()]
    kmeans_image = kmeans_image.reshape((enhanced_image.shape))
    kmeans_image = cv2.cvtColor(kmeans_image, cv2.COLOR_BGR2GRAY)
# =======================================================================

# ============================ Modified code ============================
    # Resize images
    resize_scale_w = NORM_INPUT_W_SIZE / width
    resize_scale_h = NORM_INPUT_H_SIZE / height

    resize_criteria = -1
    if resize_scale_w < resize_scale_h:
        resize_scale = resize_scale_w
        resize_criteria = 0
    else:
        resize_scale = resize_scale_h
        resize_criteria = 1

    kmeans_image = cv2.resize(kmeans_image,
                             (0, 0),
                             fx=resize_scale,
                             fy=resize_scale,
                             interpolation=cv2.INTER_CUBIC)
    label_image = cv2.resize(label_image, 
                             (0, 0),
                             fx=resize_scale,
                             fy=resize_scale,
                             interpolation=cv2.INTER_CUBIC)
 
    width = round(width * resize_scale)
    height = round(height * resize_scale)
# =======================================================================

    # Pad image
    pad_top = 0
    pad_bottom = 0
    pad_left = 0
    pad_right = 0

    if resize_criteria == 0:
        pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

        pad_top = pad_height
        pad_bottom = pad_height
        
        total_height = height + (pad_height * 2)

        if total_height > NORM_INPUT_H_SIZE:
            pad_top = pad_top - (total_height - NORM_INPUT_H_SIZE)
        elif total_height < NORM_INPUT_H_SIZE:
            pad_bottom = pad_bottom + (NORM_INPUT_H_SIZE - total_height)

    elif resize_criteria == 1:
        pad_width = int((NORM_INPUT_W_SIZE - width) / 2)

        pad_left = pad_width
        pad_right = pad_width

        total_width = width + (pad_width * 2)

        if total_width > NORM_INPUT_W_SIZE:
            pad_left = pad_left - (total_width - NORM_INPUT_W_SIZE)
        elif total_width < NORM_INPUT_W_SIZE:
            pad_right = pad_right + (NORM_INPUT_W_SIZE - total_width)

    else:
        print("Resize Error, [ocurred index] = ", idx)
        break
    
    kmeans_image = cv2.copyMakeBorder(kmeans_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    label_image = cv2.copyMakeBorder(label_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Save normalized image
    cv2.imwrite(os.path.join(val_dir, f'scar_{idx:03d}.png'), kmeans_image)
    cv2.imwrite(os.path.join(val_dir, f'label_{idx:03d}.png'), label_image)